# Data Exploration

In [3]:
%load_ext autotime

In [61]:
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import pickle

time: 5.84 ms


In [4]:
mura_path = Path('data/MURA-v1.1/')

time: 1.37 ms


In [31]:
train_paths = pd.read_csv(mura_path.joinpath('train_image_paths.csv'), header=None)
valid_paths = pd.read_csv(mura_path.joinpath('valid_image_paths.csv'), header=None)

time: 80 ms


In [34]:
def parse_path(df):
    parsed = df[0].str.split('/')
    df['body_part'] = parsed.apply(lambda x: x[2].split('_')[1].lower())
    df['patient_id'] = parsed.apply(lambda x: int(x[3].replace('patient', '')))
    df['study_id'] = parsed.apply(lambda x: int(x[4].split('_')[0].replace('study', '')))
    df['target_label'] = parsed.apply(lambda x: 0 if x[4].split('_')[1] == 'negative' else 1)
    df['image_id'] = parsed.apply(lambda x: int(x[5].split('.')[0].replace('image', '')))

time: 4.88 ms


In [36]:
parse_path(train_paths)
parse_path(valid_paths)

time: 258 ms


In [58]:
train_paths.shape

(36808, 6)

time: 4 ms


In [62]:
with mura_path.joinpath('train_groups.pkl').open('wb') as pkl_file:
    pickle.dump(train_paths, pkl_file)
with mura_path.joinpath('valid_groups.pkl').open('wb') as pkl_file:
    pickle.dump(valid_paths, pkl_file)

time: 81.6 ms


## Check for the range of number of images by study and target

In [60]:
train_paths.groupby(['body_part', 'patient_id', 'study_id', 'target_label'])[['image_id']].max().reset_index().groupby(['image_id', 'target_label']).count()

body_part  patient_id  study_id
image_id target_label                                 
1        0                   828         828       828
         1                   303         303       303
2        0                  2412        2412      2412
         1                  1321        1321      1321
3        0                  3998        3998      3998
         1                  2468        2468      2468
4        0                   951         951       951
         1                   936         936       936
5        0                    72          72        72
         1                   125         125       125
6        0                    12          12        12
         1                    15          15        15
7        0                     6           6         6
         1                     7           7         7
8        1                     2           2         2
11       0                     1           1         1

time: 49.4 ms


In [55]:
train_paths[train_paths.image_id.gt(7)]

,0,body_part,patient_id,study_id,target_label,image_id
6812,MURA-v1.1/train/XR_SHOULDER/patient02197/study...,shoulder,2197,1,0,8
6813,MURA-v1.1/train/XR_SHOULDER/patient02197/study...,shoulder,2197,1,0,9
6814,MURA-v1.1/train/XR_SHOULDER/patient02197/study...,shoulder,2197,1,0,10
6815,MURA-v1.1/train/XR_SHOULDER/patient02197/study...,shoulder,2197,1,0,11
23837,MURA-v1.1/train/XR_WRIST/patient07402/study1_p...,wrist,7402,1,1,8
23916,MURA-v1.1/train/XR_WRIST/patient07419/study1_p...,wrist,7419,1,1,8


time: 20.6 ms


In [56]:
valid_paths.groupby(['body_part', 'patient_id', 'study_id'])[['image_id']].max().reset_index().groupby('image_id').count()

,body_part,patient_id,study_id
image_id,,,
1,101,101,101
2,412,412,412
3,517,517,517
4,135,135,135
5,30,30,30
6,1,1,1
7,1,1,1
8,1,1,1
10,1,1,1


time: 27.9 ms
